In [1]:
# !wget https://raw.githubusercontent.com/pyannote/AMI-diarization-setup/5182c80724f7a21d06aad4f373f2ae6d1da9d8ea/pyannote/download_ami.sh
# !mkdir ami
# !bash download_ami.sh ami

In [2]:
from glob import glob
import numpy as np
import malaya_speech
import os
import random

`pyaudio` is not available, `malaya_speech.streaming.pyaudio_vad.stream` is not able to use.


In [3]:
ami = glob('ami/amicorpus/*/*/*.wav')
ami = {os.path.split(f)[1].replace('.wav', ''): f for f in ami}
rttm = glob('*.rttm')
len(ami), len(rttm)

(171, 2)

In [4]:
file = rttm[0]
annotations = malaya_speech.extra.rttm.load(file)

In [5]:
from tqdm import tqdm

In [18]:
labels = []
for mix in random.sample(annotations.keys(), 10):
    if mix not in ami:
        continue
    print(mix)
    sample = annotations[mix]
    y, sr = malaya_speech.load(ami[mix])
    frames = malaya_speech.generator.frames(y, 30, sr, False)

    for i in tqdm(range(len(frames))):
        if len(sample.crop(frames[i].timestamp, frames[i].timestamp + frames[i].duration)._labelNeedsUpdate):
            label = 1
        else:
            label = 0
        labels.append({
            'waveforms': frames[i].array,
            'targets': label,
        })
    break

TS3003d.Mix-Headset


100%|███████████████████████████████████| 87273/87273 [00:42<00:00, 2071.46it/s]


In [19]:
len(labels)

87273

In [21]:
import pickle

with open('vad-test.pkl', 'wb') as fopen:
    pickle.dump(labels, fopen)